# Import libraries

In [1]:
import os
import torch

from huggingface_hub import login
from datasets import load_dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments, TrainerCallback
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Set Paths and Hyperparameters

In [2]:
# Base path
base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..', '..'))

# Source and target language
source_language = "Early Modern Bohemian German"
target_language = "English"

# Translation direction
translation_direction = "DE_to_EN" if source_language == "Early Modern Bohemian German" else "EN_to_DE"

# Model parameters
unsloth_model_name = 'unsloth/gemma-2-2b-it-bnb-4bit'
company_name = 'alphabet'

model_name = unsloth_model_name.split('/')[1]
max_new_tokens = 2000           # Maximum number of model output
max_seq_length = 5000           # Maximum of input tokens
dtype = None                    # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True             # Use 4bit quantization to reduce memory usage. Can be False.

# Model output path
model_output_path = os.path.join(
    base_path, 
    'models', 
    company_name, 
    model_name,
    'finetuning',
    translation_direction)

# Finetuning prompts path
finetuning_prompts_path = os.path.join(
    base_path, 
    'data', 
    'icl_prompts',
    company_name,
    model_name,
    translation_direction, 
    'finetuning_prompt_check.txt'
)

# Print paths
print(f'Company name: {company_name}')
print(f'Model name: {model_name}')
print(f'Base path: {base_path}')
print(f'Translation direction: {translation_direction}')
print(f'Finetuning prompts path: {finetuning_prompts_path}')
print(f'Model output path: {model_output_path}')

# Hugging face login
hub_token = "hf_..."
login(hub_token, add_to_git_credential=True)

Company name: alphabet
Model name: gemma-2-2b-it-bnb-4bit
Base path: /cs/student/msc/csml/2023/ngriessh/historical_mt
Translation direction: DE_to_EN
Finetuning prompts path: /cs/student/msc/csml/2023/ngriessh/historical_mt/data/icl_prompts/alphabet/gemma-2-2b-it-bnb-4bit/DE_to_EN/finetuning_prompt_check.txt
Model output path: /cs/student/msc/csml/2023/ngriessh/historical_mt/models/alphabet/gemma-2-2b-it-bnb-4bit/finetuning/DE_to_EN
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /cs/student/msc/csml/2023/ngriessh/.cache/huggingface/token
Login successful


# Load Training and Validation Dataset

In [3]:
# Data preparation
dataset = load_dataset("niclasgriesshaber/EarlyModernGerman_to_EN_finetuning")
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

In [4]:
train_dataset

Dataset({
    features: ['Early Modern Bohemian German', 'English'],
    num_rows: 2429
})

In [5]:
validation_dataset

Dataset({
    features: ['Early Modern Bohemian German', 'English'],
    num_rows: 269
})

# Prompt template

In [6]:
prompt_template = """Translate the following from {} to {}:

### Input
{}

### Translation
{}"""

In [7]:
# Apply prompt template
def formatting_prompts_func(examples, source_language, target_language):

    source_texts = examples[source_language]
    target_texts = examples[target_language]
    texts = []

    for source_text, target_text in zip(source_texts, target_texts):
        # Format the prompt with dynamic source and target languages
        text = prompt_template.format(
            source_language, 
            target_language,
            source_text,
            target_text
        ) + '<eos>'
        texts.append(text)

    return {"text": texts}

# Apply Prompt Template to Validation and Test Dataset

In [8]:
# Apply prompt template to train dataset
train_dataset = train_dataset.map(
    lambda examples: formatting_prompts_func(examples, source_language, target_language),
    batched=True
)

# Apply prompte template to validation dataset
validation_dataset = validation_dataset.map(
    lambda examples: formatting_prompts_func(examples, source_language, target_language),
    batched=True
)

In [9]:
# Output a text file to check prompt
with open(finetuning_prompts_path, "w") as f:
    f.write(validation_dataset['text'][0])

# Load Model

In [10]:
# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = unsloth_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.575 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

# PEFT

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.1, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.9.post4 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


# Finetuning

In [12]:
# Custom Callback to Save LoRA Parameters and Track Validation Loss after Each Epoch
class SaveLoRAPeftCallback(TrainerCallback):

    def on_epoch_end(self, args, state, control, **kwargs):

        # Get epoch
        num_epoch = round(state.epoch)

        # Print end of epoch
        print(f'End of epoch {num_epoch}')

        # Evaluate the model on the validation dataset
        eval_results = trainer.evaluate()

        # Extract the validation loss
        validation_loss = eval_results["eval_loss"]

        # Print the validation loss
        print(f"Validation Loss after epoch {num_epoch}: {validation_loss}")

        # Save the validation loss to a file
        with open("validation_loss.txt", "a") as f:
            f.write(f"Validation Loss after epoch {num_epoch}: {validation_loss}\n")

        # Create folder for each epoch
        output_dir = os.path.join(args.output_dir, f"epoch_{num_epoch}")
        os.makedirs(output_dir)

        # Print saving LoRA parameters
        print(f"Saving LoRA parameters to {output_dir} at the end of epoch {round(state.epoch)}")
        
        # Save only the LoRA adapters
        kwargs['model'].save_pretrained(output_dir)

        print('Saved. Emptying torch.cuda.empty_cache() now.')
        torch.cuda.empty_cache()

# Initialize the trainer with the callback
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        eval_strategy="epoch",
        save_strategy="no",
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir=model_output_path,
        seed=0,
        log_level="info",
        report_to="none",
    ),
    callbacks=[SaveLoRAPeftCallback()],
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map (num_proc=2):   0%|          | 0/2429 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/269 [00:00<?, ? examples/s]

Using auto half precision backend


In [13]:
# Run evaluation at the beginning of the first epoch
print('Evaluation at the beginning of epoch 1')
trainer.evaluate()

Evaluation at the beginning of epoch 1



***** Running Evaluation *****
  Num examples = 269
  Batch size = 1
AUTOTUNE bmm(8x105x256, 8x256x105)
  triton_bmm_1 0.0085 ms 100.0%
  triton_bmm_2 0.0102 ms 82.8%
  triton_bmm_7 0.0102 ms 82.8%
  triton_bmm_3 0.0113 ms 75.3%
  triton_bmm_6 0.0123 ms 69.2%
  bmm 0.0123 ms 69.0%
  triton_bmm_4 0.0123 ms 69.0%
  triton_bmm_5 0.0143 ms 59.2%
  triton_bmm_10 0.0143 ms 59.2%
  triton_bmm_9 0.0154 ms 55.2%
SingleProcess AUTOTUNE benchmarking takes 2.2084 seconds and 0.0272 seconds precompiling
AUTOTUNE bmm(8x105x105, 8x105x256)
  bmm 0.0082 ms 100.0%
  triton_bmm_21 0.0082 ms 100.0%
  triton_bmm_22 0.0082 ms 100.0%
  triton_bmm_23 0.0082 ms 100.0%
  triton_bmm_26 0.0082 ms 100.0%
  triton_bmm_25 0.0092 ms 88.9%
  triton_bmm_19 0.0102 ms 80.0%
  triton_bmm_20 0.0102 ms 80.0%
  triton_bmm_24 0.0102 ms 80.0%
  triton_bmm_33 0.0102 ms 80.0%
SingleProcess AUTOTUNE benchmarking takes 2.0748 seconds and 0.0009 seconds precompiling


{'eval_loss': 4.953486442565918,
 'eval_model_preparation_time': 0.0107,
 'eval_runtime': 34.5142,
 'eval_samples_per_second': 7.794,
 'eval_steps_per_second': 7.794}

In [14]:
# Start finetuning
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,429 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 1,821
 "-____-"     Number of trainable parameters = 41,533,440
AUTOTUNE bmm(8x239x239, 8x239x256)
  bmm 0.0123 ms 100.0%
  triton_bmm_119 0.0225 ms 54.5%
  triton_bmm_114 0.0246 ms 50.0%
  triton_bmm_128 0.0256 ms 48.0%
  triton_bmm_117 0.0266 ms 46.2%
  triton_bmm_121 0.0266 ms 46.2%
  triton_bmm_120 0.0287 ms 42.9%
  triton_bmm_124 0.0307 ms 40.0%
  triton_bmm_127 0.0307 ms 40.0%
  triton_bmm_123 0.0328 ms 37.5%
SingleProcess AUTOTUNE benchmarking takes 2.1119 seconds and 0.0008 seconds precompiling
AUTOTUNE bmm(8x239x256, 8x256x239)
  triton_bmm_139 0.0133 ms 100.0%
  bmm 0.0154 ms 86.7%
  triton_bmm_135 0.0154 ms 86.7%
  triton_bmm_143 0.0154 ms 86.7%
  triton_bmm_146 0.0156 ms 85.4%
  triton_bmm_142 0.0162 ms 82.4%
  triton_bmm_138 0.0164 ms 

Epoch,Training Loss,Validation Loss,Model Preparation Time
0,2.084500,2.392666,0.010700
1,1.973900,2.256228,0.010700
2,1.664600,2.324682,0.010700


AUTOTUNE bmm(8x256x256, 8x256x256)
  bmm 0.0174 ms 100.0%
  triton_bmm_233 0.0236 ms 73.9%
  triton_bmm_228 0.0256 ms 68.0%
  triton_bmm_242 0.0263 ms 66.2%
  triton_bmm_231 0.0276 ms 63.0%
  triton_bmm_235 0.0276 ms 63.0%
  triton_bmm_234 0.0297 ms 58.6%
  triton_bmm_238 0.0317 ms 54.8%
  triton_bmm_241 0.0317 ms 54.8%
  triton_bmm_237 0.0348 ms 50.0%
SingleProcess AUTOTUNE benchmarking takes 2.2861 seconds and 0.0010 seconds precompiling
AUTOTUNE bmm(8x256x256, 8x256x256)
  bmm 0.0113 ms 100.0%
  triton_bmm_253 0.0133 ms 84.6%
  triton_bmm_249 0.0154 ms 73.3%
  triton_bmm_257 0.0154 ms 73.3%
  triton_bmm_260 0.0154 ms 73.3%
  triton_bmm_261 0.0160 ms 70.4%
  triton_bmm_252 0.0162 ms 69.4%
  triton_bmm_256 0.0164 ms 68.8%
  triton_bmm_250 0.0181 ms 62.2%
  triton_bmm_247 0.0184 ms 61.1%
SingleProcess AUTOTUNE benchmarking takes 2.1932 seconds and 0.0011 seconds precompiling
AUTOTUNE bmm(8x256x256, 8x256x256)
  triton_bmm_269 0.0143 ms 100.0%
  bmm 0.0164 ms 87.5%
  triton_bmm_273 0.01

End of epoch 1
Validation Loss after epoch 1: 2.3926663398742676
Saving LoRA parameters to /cs/student/msc/csml/2023/ngriessh/historical_mt/models/alphabet/gemma-2-2b-it-bnb-4bit/finetuning/DE_to_EN/epoch_1 at the end of epoch 1


loading configuration file config.json from cache at /cs/student/msc/csml/2023/ngriessh/.cache/huggingface/hub/models--unsloth--gemma-2-2b-it-bnb-4bit/snapshots/cfc3a78e491db11ed6fd461d9ce8ba921de01602/config.json
Model config Gemma2Config {
  "_name_or_path": "unsloth/gemma-2-2b-it",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": [
    1,
    107
  ],
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 8,
  "num_hidden_layers": 26,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_c

...


***** Running Evaluation *****
  Num examples = 269
  Batch size = 1


End of epoch 3
Validation Loss after epoch 3: 2.3246822357177734
Saving LoRA parameters to /cs/student/msc/csml/2023/ngriessh/historical_mt/models/alphabet/gemma-2-2b-it-bnb-4bit/finetuning/DE_to_EN/epoch_3 at the end of epoch 3


loading configuration file config.json from cache at /cs/student/msc/csml/2023/ngriessh/.cache/huggingface/hub/models--unsloth--gemma-2-2b-it-bnb-4bit/snapshots/cfc3a78e491db11ed6fd461d9ce8ba921de01602/config.json
Model config Gemma2Config {
  "_name_or_path": "unsloth/gemma-2-2b-it",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": [
    1,
    107
  ],
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 8,
  "num_hidden_layers": 26,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_c

Saved. Emptying torch.cuda.empty_cache() now.


: 